<a href="https://colab.research.google.com/github/talaricoferreira/mestrado/blob/main/Classificador_Baysiano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Testes com Algoritimo de Baysiana tirada do Artigo de Holmes ans Held (2006)
[Implementação em R do Artigo](https://rstudio-pubs-static.s3.amazonaws.com/208180_b659633007eb45aa9c48e4c50b8afc07.html)



---



##Importando Bibliotecas

In [54]:
import numpy as np
from scipy.stats import truncnorm, multivariate_normal
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from collections import Counter


## DEFINIDO AS FUNÇOES

###Holmes and Held Extension Multiclass

In [55]:


def gibbs_sampling(X, y, N_sim, burn_in):
    D = X.shape[1]
    N = len(X)
    y_len = len(y)

    # Matrix projeção coeficiente aleatória
    V = np.random.rand(D, D)
    cov_X = np.cov(X.T)
    V = cov_X + 1e-12 * np.eye(D)
    S = np.dot(V, X.T)

    h, w, u = np.zeros((3, N))
    for j in range(N):
        h[j] = X[j, :] @ S[:, j]
        w[j] = h[j] / (1 - h[j])
        u[j] = w[j] + 1

    z = np.where(y == 0, truncnorm.rvs(-np.inf, 0, loc=0, scale=1, size=y_len),
                 truncnorm.rvs(0, np.inf, loc=0, scale=1, size=y_len))

    theta_chain_holmes = np.zeros((N_sim, D))
    M = S @ z

    for t in range(1, N_sim + 1):
        for j in range(N):
            z_old = z[j]
            m = X[j, :] @ M
            m = m - w[j] * (z[j] - m)
            if y[j] == 0:
                z[j] = truncnorm.rvs(-np.inf, 0, loc=m, scale=abs(u[j]), size=1)
            else:
                z[j] = truncnorm.rvs(0, np.inf, loc=m, scale=abs(u[j]), size=1)
            M = M + (z[j] - z_old) * S[:, j]
        if t >= burn_in:
            theta_chain_holmes[t - burn_in, :] = multivariate_normal.rvs(mean=M, cov=V, size=1)

    post_theta_holmes = np.mean(theta_chain_holmes[burn_in:], axis=0)

    return post_theta_holmes


### Holmes and Held Extension Multiclass

In [56]:


def gibbs_sampling_multiclass(X, y, N_sim, burn_in):
    D = X.shape[1]
    N = len(X)
    C = len(np.unique(y))  # Número de classes

    V = np.random.rand(D, D)
    cov_X = np.cov(X.T)
    V = cov_X + 1e-12 * np.eye(D)
    S = np.dot(V, X.T)

    h, w, u = np.zeros((3, N))
    for j in range(N):
        h[j] = X[j, :] @ S[:, j]
        w[j] = h[j] / (1 - h[j])
        u[j] = w[j] + 1

    theta_chain_holmes = np.zeros((N_sim, D, C))
    M = np.zeros((D, C))

    for t in range(1, N_sim + 1):
        for c in range(C):
            z = np.where(y == c, truncnorm.rvs(0, np.inf, loc=0, scale=1, size=N),
                         truncnorm.rvs(-np.inf, 0, loc=0, scale=1, size=N))
            for j in range(N):
                z_old = z[j]
                m = X[j, :] @ M[:, c]
                m = m - w[j] * (z[j] - m)
                z[j] = truncnorm.rvs(-np.inf, np.inf, loc=m, scale=abs(u[j]), size=1)
                M[:, c] = M[:, c] + (z[j] - z_old) * S[:, j]
            if t >= burn_in:
                theta_chain_holmes[t - burn_in, :, c] = multivariate_normal.rvs(mean=M[:, c], cov=V, size=1)

    post_theta_holmes = np.mean(theta_chain_holmes[burn_in:], axis=0)

    return post_theta_holmes


### Preditor

In [57]:
def predictor(X, post_theta):
    predictions = np.dot(X, post_theta)
    return np.argmax(predictions, axis=1)

##Usando o Dataset Iris como Teste
Por ser uma base de dados conhecida é mais facil usa-la para testar o algoritimo, aqui já fazendo o *split* da base


---

E após isso rodando o gibbs_sampling

In [58]:
iris = load_iris()
X = iris.data
y = iris.target
target_names = iris.target_names

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
# Executar o Gibbs sampling para obter os parâmetros estimados
N_sim = 1000
burn_in = 200
post_theta_holmes = gibbs_sampling_multiclass(X_train, y_train, N_sim, burn_in)

print(post_theta_holmes)

[[-0.02931736  0.21801698 -0.35727082]
 [ 0.27799136 -0.5497408   0.21835314]
 [-0.24879957 -0.00607172  0.18006429]
 [ 0.01262434 -0.58362922  0.56794001]]


In [ ]:
df = pd.read_csv('/content/dataframe_violins.csv')

In [ ]:
anonym_synonym = ['Anonyme','ok\xc3\xa4nd/os\xc3\xa4ker (unknown or not sure)','Auteur onbekend',\
                  'Anonymous','anonimo']

for l in anonym_synonym:
    df.loc[df.Maker==l,'Maker'] = np.nan

df.loc[df.Country=='Unclear','Country'] = np.nan
df.loc[(df.Year<500),'Year'] = np.nan; df.loc[(df.Year>2017),'Year'] = np.nan


makersorg = Counter(df.Maker.loc[~df.Maker.isnull()]).most_common()

makers = makersorg[0:7]
makers.append(makersorg[9])
names_makers = [name[0] for name in makers]

In [ ]:
df2 = df[df['Maker'].isin(names_makers)]

In [ ]:
variables = ['a','b','c','d','e','f','h1','h2','width','s1','s2','s3','s4','s5','s6','H']

M = df2.loc[:,variables]
M = (M - M.mean(axis=0))/(M.std(axis=0))
N
##Matriz com as features de cada luthier
M = M.dropna() #M.get_values() para formato numpy

##PCA para plotar
X = PCA(n_components=2).fit_transform(M)

###Teste de acurácia

In [59]:
# Fazer previsões nos dados de teste
y_pred = predictor(X_test, post_theta_holmes)
y_test_mapped = iris.target_names[y_test]
# Calcular a acurácia do preditor
print(y_test_mapped)
print()
print(y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do preditor:", accuracy)

['versicolor' 'setosa' 'virginica' 'versicolor' 'versicolor' 'setosa'
 'versicolor' 'virginica' 'versicolor' 'versicolor' 'virginica' 'setosa'
 'setosa' 'setosa' 'setosa' 'versicolor' 'virginica' 'versicolor'
 'versicolor' 'virginica' 'setosa' 'virginica' 'setosa' 'virginica'
 'virginica' 'virginica' 'virginica' 'virginica' 'setosa' 'setosa'
 'setosa' 'setosa' 'versicolor' 'setosa' 'setosa' 'virginica' 'versicolor'
 'setosa' 'setosa' 'setosa' 'virginica' 'versicolor' 'versicolor' 'setosa'
 'setosa' 'versicolor' 'virginica' 'virginica' 'versicolor' 'virginica'
 'versicolor' 'virginica' 'versicolor' 'setosa' 'virginica' 'versicolor'
 'setosa' 'setosa' 'setosa' 'versicolor']

[2 0 2 2 2 0 2 2 2 2 2 0 0 0 0 2 2 2 2 2 0 2 0 2 2 2 2 2 0 0 0 0 2 0 0 2 2
 0 0 0 2 2 2 0 0 2 2 2 2 2 2 2 2 0 2 2 0 0 0 2]
Acurácia do preditor: 0.6833333333333333
